<a href="https://colab.research.google.com/github/nsriniva/DS-Unit-2-Build/blob/main/OnlineNewsPopularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/nsriniva/DS-Unit-2-Build/master/'
    '''
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap
    '''
# If you're working locally:
else:
    DATA_PATH = '../data/'

from collections import OrderedDict
from math import isclose
import zipfile 
from urllib.request import urlopen
import io

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import chi2_contingency

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LinearRegression, LogisticRegression, \
                                 LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

try:
  from category_encoders import OrdinalEncoder, OneHotEncoder
except:
  !pip install category_encoders==2.*
  from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
try:
  import eli5
except:
  !pip install eli5
  import eli5

from eli5.sklearn import PermutationImportance
try:
  import shap
except:
  !pip install shap
  import shap

try:
  from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
except:
  !pip install pdpbox
  from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot


# For details about the data cleanup, please see 
# https://github.com/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/CleanupOnlineNewsPopularity.ipynb
# and 'The Dataset' section of
# https://nsriniva.github.io/2020-10-23-DSPT9-Unit1-BuildProject/

# Cleaned up and uploaded csv data file from 
# https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip 
# in
# https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity
# to my github repo as
# https://github.com/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/OnlineNewsPopularity.csv.zip?raw=true

# The associated names file is available at
# https://raw.githubusercontent.com/nsriniva/DS-Unit-2-Applied-Modeling/master/OnlineNewsPopularity.names


onp_url = DATA_PATH + 'OnlineNewsPopularity.csv.zip'

onp_df = pd.read_csv(onp_url, compression='zip')

null_values = onp_df.isna().sum().sum()

print(f"There are {['','no'][int(null_values==0)]} invalid values in the dataset!")

# The zscore() method from the scipy.stats package is used to compute z scores 
# for the shares values. These z scores is compared against the specified  
# sigma value to generate a boolean filter array that could be used to 
# paritition the dataset based on whether the zscore is greater than the
# specified sigma.
def get_sigma_filter(df, sigma=0.5):
  z = np.abs(stats.zscore(df.shares))
  return np.where(z>sigma)[0]

# Use the boolean filter array provided by get_sigma_filter() to
# ignore entries with zscore greater than 0.5 and compute the
# median and max 'shares' values for the remaining entries.
def classification_marks(df):

  shares_info = df.drop(get_sigma_filter(df)).shares

  max = shares_info.max()
  median = shares_info.median()

  return median, max


shares_median = onp_df.shares.median()

print(shares_median)
# Use the medium(median) value to classify articles into 
# unpopular(0) and popular(1)  
onp_df['popularity'] = onp_df.shares.apply(lambda x: 0 if x < shares_median else 1)

display(onp_df.shape)

# Remove outliers
def remove_outliers(df, sigma=0.5):
  df = df.copy()
  return df.drop(get_sigma_filter(df, sigma))


onp_no_df = onp_df.copy()

#onp_no_df = remove_outliers(onp_no_df, 0.25)

shares_median = onp_no_df.shares.median()

print(shares_median)

# Use the medium(median) value to classify articles into 
# unpopular(0) and popular(1)  
onp_no_df['popularity'] = onp_no_df.shares.apply(lambda x: 0 if x < shares_median else 1)

display(onp_no_df.shape)



# The baseline accuracy or the value we'd get by just guessing that that the
# value is always the majority class

target = 'popularity'

baseline_accuracy = onp_no_df[target].value_counts(normalize=True).max()

print(f'baseline_accuracy = {baseline_accuracy:0.4f}')

# Drop the 'shares' column used to derive 'popularity' along
# with the non predictive 'url' and 'timedelta' columns.
drop_cols = ['shares', 'url', 'timedelta']

onp_no_df = onp_no_df.drop(columns=drop_cols)
# Will use a random split of 64% Training, 16% Validation and 20% Test 

X = onp_no_df.drop(columns=target)
y = onp_no_df[target]

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,train_size=0.8, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size=0.8, random_state=42)

display(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

display(y_train.value_counts(normalize=True))
baseline_accuracy = y_train.value_counts(normalize=True).max()

print(f'baseline_accuracy = {baseline_accuracy:0.4f}')




/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of th

There are no invalid values in the dataset!
1400.0


(39644, 51)

1400.0


(39644, 51)

...

(25372,)

(6343,)

(7929,)

1    0.536891
0    0.463109
Name: popularity, dtype: float64

baseline_accuracy = 0.5369


In [2]:
def model_acc_auc(model, *Xyn, names=None):

  if names is None:
    names = ['Training', 'Validation', 'Test']
  p = Xyn
  n = len(p)

  assert n >= 2
  assert n%2 == 0

  n = n//2
  Xn = []
  yn = []
  for i in range(0,n):
    Xn.append(p[2*i])
    yn.append(p[2*i+1])

  accn = []
  aucn = []
  bl = []
  for i,X in enumerate(Xn):
    bl.append(yn[i].value_counts(normalize=True).max())

    accn.append(model.score(X, yn[i]))
    
    y_proba = model.predict_proba(X)[:,-1]
    aucn.append(roc_auc_score(yn[i], y_proba))
  
  for i, acc in enumerate(zip(accn, bl, aucn)):
    name = names[i]
    print(f'{name} Accuracy : {acc[0]:0.4f}/{acc[1]:0.4f}/{acc[2]:0.4f}')

In [ ]:
# For parameter k, use SelectKBest to compute the k best
# features and use those to train a LogisticRegressionCV
# model.
def select_and_fit(k, X_tr, y_tr, X_v, y_v):
    
  selector = SelectKBest(score_func=f_classif, k=k)
  X_train_selected = selector.fit_transform(X_tr, y_train)
  X_val_selected = selector.transform(X_v)

  model = LogisticRegressionCV()
  model.fit(X_train_selected, y_tr)
  
  return model.score(X_val_selected, y_v), model, selector

def get_best_k_model(X_tr, y_tr, X_v, y_v):
  best_model = None
  best_selector = None
  best_features=[]
  best_k = 0
  best_acc = 0

  # n = 62
  n = X_tr.shape[1]
  # Loop through k and compare accuracies to determine the best
  # k features(best_features) with the highest accuracy
  # One run with k from 1 - 62(range(1,n+1)) gave the best k as 51 - in order to reduce
  # the time looking for best k, we just run once with k=51
  for k in range(51, 52):
      acc, model, selector = select_and_fit(k, X_tr, y_tr, X_v, y_v)
      #print(acc, feat)
      if acc > best_acc:
        best_acc = acc
        best_k = k
        best_model = model
        best_selector = selector

  print(f'best_k = {best_k}\nbest Accuracy = {best_acc:0.4f}\n')

  return best_acc, best_k, best_selector, best_model

encoder = OneHotEncoder(use_cat_names=True)

X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train_encoded)
X_val_transformed = scaler.transform(X_val_encoded)
X_test_transformed = scaler.transform(X_test_encoded)

best_acc, best_k, best_selector, best_model = get_best_k_model(X_train_transformed, y_train, X_val_transformed, y_val)

model_acc_auc(best_model, best_selector.transform(X_train_transformed), y_train, best_selector.transform(X_val_transformed), y_val, best_selector.transform(X_test_transformed), y_test)


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,30))


best_features = X_train_encoded.columns[best_selector.get_support()]
# Get and plot model coefficients.
coefficients = pd.Series(best_model.coef_[0], best_features)
coefficients.sort_values().plot.barh(); #bar charts


In [ ]:
# Simple model, with OrdinalEncoder for the data_channel and weekday categorical
# columns and a DecisionTreeClassifier with default parameter values.
model = make_pipeline(
  OrdinalEncoder(), 
  DecisionTreeClassifier()
)

model.fit(X_train, y_train)

display(y_train.value_counts(normalize=True))
display(y_val.value_counts(normalize=True))

training_bl = y_train.value_counts(normalize=True).max()
validation_bl = y_val.value_counts(normalize=True).max()
test_bl = y_test.value_counts(normalize=True).max()
 
model_acc_auc(model, X_train, y_train, X_val, y_val)

transformers = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)

model = RandomForestClassifier(n_estimators=103, random_state=42, n_jobs=-1, max_depth=25, min_samples_leaf=3, max_features=0.3)
model.fit(X_train_transformed, y_train)

model_acc_auc(model, X_train_transformed, y_train, X_val_transformed, y_val)

permuter = PermutationImportance(
    model, 
    scoring='accuracy', 
    n_iter=5, 
    random_state=42
)

permuter.fit(X_val_transformed, y_val)


In [ ]:
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, # No limit: show permutation importances for all features
    feature_names=feature_names # must be a list
)

In [ ]:
'''
print('Shape before removing', X_train.shape)

minimum_importance = 0 
mask = permuter.feature_importances_ > minimum_importance
features = X_train.columns[mask]
X_train = X_train[features]

print('Shape after removing ', X_train.shape)

X_val = X_val[features]
X_test = X_test[features]

'''

In [ ]:
model = make_pipeline(
  OrdinalEncoder(), 
  DecisionTreeClassifier(max_depth=7,random_state=42, min_samples_leaf=3)
)

model.fit(X_train, y_train)

display(y_train.value_counts(normalize=True))
display(y_val.value_counts(normalize=True))

model_acc_auc(model, X_train, y_train, X_val, y_val)



In [ ]:
pipe = make_pipeline(
    OrdinalEncoder(), 
    RandomForestClassifier(n_estimators=103, random_state=42, n_jobs=-1, max_depth=25, min_samples_leaf=3, max_features=0.3)
)

# Fit on train, score on val
pipe.fit(X_train, y_train)

model_acc_auc(pipe, X_train, y_train, X_val, y_val, X_test, y_test)


In [ ]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

eval_set = [(X_train_encoded, y_train), 
            (X_val_encoded, y_val)]

model = XGBClassifier(n_estimators=1000, random_state=42, n_jobs=-1, max_depth=6, learning_rate=0.5) 

eval_metric = 'auc'
model.fit(X_train_encoded, y_train, 
          eval_set=eval_set, 
          eval_metric=eval_metric,
          early_stopping_rounds=400)


In [ ]:
results = model.evals_result()
train_error = results['validation_0'][eval_metric]
val_error = results['validation_1'][eval_metric]
epoch = list(range(1, len(train_error)+1))
plt.plot(epoch, train_error, label='Train')
plt.plot(epoch, val_error, label='Validation')
plt.ylabel(f'Classification {eval_metric.capitalize()}')
plt.xlabel('Model Complexity (n_estimators)')
plt.title('Validation Curve for this XGBoost model')
#plt.ylim((0.18, 0.22)) # Zoom in
plt.legend();

In [ ]:
model_acc_auc(model, X_train_encoded, y_train, X_val_encoded, y_val, X_test_encoded, y_test)


In [ ]:
features = ['is_weekend', 'kw_avg_avg']

print(X_train_encoded.shape, X_val_encoded.shape)

isolated = []

for feature in features:
  isolated.append(
      pdp_isolate(
      model=model, 
      dataset=X_val_encoded, 
      model_features=X_val_encoded.columns, 
      feature=feature
    )
  )

interaction = pdp_interact(
    model=model, 
    dataset=X_val_encoded, 
    model_features=X_val_encoded.columns,
    features=features
)

In [ ]:
for idx,elem in enumerate(isolated):
  pdp_plot(elem, feature_name=features[idx]);
  if features[idx] == 'is_weekend':
    # Manually change the xticks labels
    plt.xticks([0, 1], ['False', 'True']);

pdp = interaction.pdp.pivot_table(
    values='preds', 
    columns=features[0], # First feature on x axis
    index=features[1]    # Next feature on y axis
)[::-1]  # Reverse the index order so y axis is ascending

pdp = pdp.rename(columns={0:'False', 1:'True'})
plt.figure(figsize=(10,8))
sns.heatmap(pdp, annot=True, fmt='.3f', cmap='viridis')
plt.title('Partial Dependence of Article Popularity on is_weekend & kw_avg_avg');


In [ ]:

df = pd.DataFrame({
    'pred_proba': model.predict_proba(X_test_encoded)[:, -1] , 
    'popular': y_test
}) 

df= df.merge(X_test_encoded, left_index=True, right_index=True)

popular = df.popular == 1
unpopular = df.popular == 0
popular_right = (popular) == (df['pred_proba'] > 0.50)
unpopular_right = (unpopular) == (df['pred_proba'] <= 0.50)


popular_correct = df[popular&popular_right]
unpopular_correct = df[unpopular&unpopular_right]

correct = df[popular&popular_right | unpopular&unpopular_right]

display(popular_correct.shape, unpopular_correct.shape, correct.shape, df.shape)
display(correct.sample(n=10, random_state=1).sort_values(by='pred_proba'))

row_popular = X_test_encoded.loc[[25641]]

row_unpopular = X_test_encoded.loc[[5864]]

display(row_popular, row_unpopular)

In [ ]:
explainer = shap.TreeExplainer(model)

shap.initjs()

shap_values = explainer.shap_values(row_popular)
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row_popular,
    #matplotlib=True, # This does not work if link is set to 'logit'
    link='logit' # For classification, this shows predicted probabilities
)

  
  

In [ ]:
shap.initjs()

shap_values = explainer.shap_values(row_unpopular)
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row_unpopular,
    #matplotlib=True, # This does not work if link is set to 'logit'
    link='logit' # For classification, this shows predicted probabilities
)